In [11]:
import pandas as pd
import numpy as np
import re
import os
import sys
import json
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
TOP_PATH = os.environ['PWD']

In [14]:
sys.path.append(TOP_PATH + '/config')
sys.path.append(TOP_PATH + '/src')

In [15]:
from trends import *
import etl
from regression import regression

In [16]:
test_date = '10-11-2018'
test_date_2 = '11-12-2018'

In [17]:
df_2017 = pd.read_csv(TOP_PATH + '/data/cleaned/2017_cleaned.csv')

In [18]:
df_2018_2019 = pd.read_csv(TOP_PATH + '/data/cleaned/2018-2019_cleaned.csv')

In [19]:
df_2014 = pd.read_csv(TOP_PATH + '/data/cleaned/2014_cleaned.csv')

In [20]:
df_2015 = pd.read_csv(TOP_PATH + '/data/cleaned/2015_cleaned.csv')

In [21]:
df_2016 = pd.read_csv(TOP_PATH + '/data/cleaned/2016_cleaned.csv')

In [22]:
df_2018_2019['subject_race'].unique()

array(['White', 'Hispanic/Latino/a', 'Black/African American',
       'Native American', 'Middle Eastern or South Asian', 'Asian',
       'Pacific Islander'], dtype=object)

In [23]:
#DACA example for regression
# [2017,7,5] to [2017,11,5]
start_date = pd.to_datetime('07-05-2017')
end_date = pd.to_datetime('11-05-2017')
event_date = pd.to_datetime('09-05-2017')
df_regr = df_2017.copy()
df_regr = df_regr[[start_date <= pd.to_datetime(x) and end_date >= pd.to_datetime(x) for x in df_regr['date_stop']]]

In [24]:
df_regr_hisp = df_regr.copy()

In [25]:
df_regr_hisp['Hispanic'] = [1 if x == 'Hispanic/Latino/a' else 0 for x in df_regr_hisp['subject_race']]

In [26]:
daca_df = pd.read_csv(TOP_PATH + '/data/raw/DACA.csv')

In [27]:
daca_df.drop('Unnamed: 0', inplace = True, axis = 1)

In [28]:
temp = df_regr_hisp[['date_stop', 'Hispanic']].groupby('date_stop').mean().reset_index()

In [29]:
daca_df.reset_index()

,index,date,value
0,0,2017-07-05,<1
1,1,2017-07-06,1
2,2,2017-07-07,<1
3,3,2017-07-08,<1
4,4,2017-07-09,<1
...,...,...,...
119,119,2017-11-01,1
120,120,2017-11-02,2
121,121,2017-11-03,1
122,122,2017-11-04,1


In [30]:
daca_df['value'] = [int(x) if x.isnumeric() else 0 for x in daca_df['value']]

In [31]:
df = temp.merge(daca_df, left_on = 'date_stop', right_on = 'date')

In [32]:
df = df.sort_values('value').reset_index(drop = True)

In [33]:
reg = LinearRegression().fit(df['value'].values.reshape(-1,1), df['Hispanic'].values)

In [34]:
df

,date_stop,Hispanic,date,value
0,2017-07-05,0.308989,2017-07-05,0
1,2017-08-05,0.275000,2017-08-05,0
2,2017-08-06,0.246231,2017-08-06,0
3,2017-07-30,0.251397,2017-07-30,0
4,2017-08-21,0.421941,2017-08-21,0
...,...,...,...,...
118,2017-09-01,0.353846,2017-09-01,19
119,2017-09-07,0.321678,2017-09-07,37
120,2017-09-04,0.330961,2017-09-04,44
121,2017-09-06,0.302872,2017-09-06,79


In [35]:
reg.coef_

array([0.00051768])

In [36]:
reg.intercept_

0.29590542642929457

In [ ]:
x = regression(('07-05-2017', '11-05-2017'), 'Hispanic/Latino/a', 'DACA', 'log')

In [ ]:
x.coef_